In [1]:
from sklearn.datasets import load_iris
iris = load_iris()
dir(iris)

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [6]:
import pandas as pd
data = pd.DataFrame(data = iris.data, columns = iris.feature_names)
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [7]:
data['flower'] = iris.target
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [15]:
data['flower'] = data['flower'].apply(lambda x: iris.target_names[x])
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.2)

In [20]:
# first trying SVM model for classification
from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', C = 30, gamma = 'auto')
svm.fit(x_train, y_train)
svm.score(x_test, y_test)

0.9333333333333333

In [22]:
# perform cross validation using cross_val_score with differrent hyper parameters of SVM instead of another algorithm
from sklearn.model_selection import cross_val_score
cross_val_score(SVC(kernel = 'rbf', C = 30, gamma = 'auto'), iris.data, iris.target, cv = 5)

array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ])

In [23]:
cross_val_score(SVC(kernel = 'linear', C = 10, gamma = 'auto'), iris.data, iris.target, cv = 5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [24]:
cross_val_score(SVC(kernel = 'rbf', C = 10, gamma = 'auto'), iris.data, iris.target, cv = 5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [25]:
cross_val_score(SVC(kernel = 'rbf', C = 20, gamma = 'auto'), iris.data, iris.target, cv = 5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [ ]:
# we can see that there are many hyper parameters and selecting the optimum value for each hyper parameter using cross_val_score
# is manual and time taking

In [26]:
# another option is we can use a for loop
import numpy as np
kernel = ['rbf', 'linear']
C_list = [1,10,20]
avg_scores = {}
for k in kernel:
    for C in C_list:
        cv_scores = cross_val_score(SVC(kernel = k, C = C, gamma = 'auto'), iris.data, iris.target, cv = 5)
        avg_scores[k+"_"+str(C)] = np.mean(cv_scores)
avg_scores

# but again if we need to tune more hyper parameters, then for loop is not convenient
# for this sklearn provides GridSearchCV that does exactly the same thing as this for loop

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [31]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma = 'auto'), {
    "C" : [1,10,20],
    "kernel" : ['rbf', 'linear']
}, cv = 5, return_train_score = False)

clf.fit(iris.data, iris.target)
clf.cv_results_
# cv_results_ are not easy to view
# we can convert them into dataframe

{'mean_fit_time': array([0.00120068, 0.00139394, 0.0014111 , 0.00100064, 0.00140109,
        0.00160236]),
 'std_fit_time': array([4.01856761e-04, 4.95310948e-04, 5.02445162e-04, 3.01578299e-07,
        8.00418911e-04, 4.90039806e-04]),
 'mean_score_time': array([0.00100088, 0.00060449, 0.00079479, 0.0004003 , 0.00100594,
        0.00079985]),
 'std_score_time': array([1.97758115e-06, 4.93591221e-04, 3.97543935e-04, 4.90271296e-04,
        1.03523093e-05, 7.48698806e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

In [32]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001201,4.018568e-04,0.001001,0.000002,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001394,4.953109e-04,0.000604,0.000494,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001411,5.024452e-04,0.000795,0.000398,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001001,3.015783e-07,0.000400,0.000490,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001401,8.004189e-04,0.001006,0.000010,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001602,4.900398e-04,0.000800,0.000749,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [33]:
df[['param_C','param_kernel','mean_test_score']]
# based on the result we can say that we can supply the top three parameter values to get the best performance

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [34]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',

In [36]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [ ]:
# the issue with GridSearchCV is the computation cost 
# so sklearn has another class called RandomizedSearchCV
# RandomizedSearchCV does not perform all permutation and combination for different values of hyperparameters but it tries
# random combinations of hyperparameter values

In [55]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma = 'auto'), {
    "C":[1,10,20],
    "kernel":["rbf", "linear"]
}, cv = 5, return_train_score = False, n_iter = 2)

In [56]:
rs.fit(iris.data, iris.target)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='auto', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=2, n_jobs=None,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [42]:
rs.cv_results_

{'mean_fit_time': array([0.0013999 , 0.00160561]),
 'std_fit_time': array([0.00049132, 0.00049467]),
 'mean_score_time': array([0.00100236, 0.00060091]),
 'std_score_time': array([1.65456310e-06, 4.90642465e-04]),
 'param_kernel': masked_array(data=['rbf', 'rbf'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[10, 1],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'rbf', 'C': 10}, {'kernel': 'rbf', 'C': 1}],
 'split0_test_score': array([0.96666667, 0.96666667]),
 'split1_test_score': array([1., 1.]),
 'split2_test_score': array([0.96666667, 0.96666667]),
 'split3_test_score': array([0.96666667, 0.96666667]),
 'split4_test_score': array([1., 1.]),
 'mean_test_score': array([0.98, 0.98]),
 'std_test_score': array([0.01632993, 0.01632993]),
 'rank_test_score': array([1, 1])}

In [45]:
result = pd.DataFrame(rs.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001400,0.000491,0.001002,0.000002,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.0,0.966667,0.966667,1.0,0.98,0.01633,1
1,0.001606,0.000495,0.000601,0.000491,rbf,1,"{'kernel': 'rbf', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.98,0.01633,1


In [46]:
result[['param_kernel','param_C','mean_test_score']]

,param_kernel,param_C,mean_test_score
0,rbf,10,0.98
1,rbf,1,0.98


In [64]:
# now if we want to select the best algorithm with best hyper parameters
# we will select from RandomForestClassifier and LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

model_params = {
    'svm_model':{
        'model':SVC(gamma='auto'),
        'params': {
            'C' : [1,10,20],
            'kernel' : ['rbf','linear']
        }
    },
    'random_forest_model':{
        'model' : RandomForestClassifier(),
        'params' : {'n_estimators':[1,5,10]}
    },
    'logistic_regression_model':{
        'model': LogisticRegression(),
        'params' : {'C':[1,5,10]}
    }
}

In [67]:
scores = []
for model, mod_param in model_params.items():
    print(model)
    print(mod_param)
    clf = GridSearchCV(mod_param['model'],mod_param['params'], cv = 5, return_train_score = False)
    clf.fit(iris.data, iris.target)
    #print(clf.best_estimator_)
    scores.append({'model': model,
                  'best_score': clf.best_score_,
                  'best_params': clf.best_params_})
scores  

svm_model
{'model': SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'params': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}}
random_forest_model
{'model': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'params': {'n_estimators': [1, 5, 10]}}
logistic_regression_model
{'model': LogisticRegressio

c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scik

[{'model': 'svm_model',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 1, 'kernel': 'rbf'}},
 {'model': 'random_forest_model',
  'best_score': 0.9666666666666668,
  'best_params': {'n_estimators': 5}},
 {'model': 'logistic_regression_model',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 10}}]

In [68]:
df = pd.DataFrame(data = scores)
df

,model,best_score,best_params
0,svm_model,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest_model,0.966667,{'n_estimators': 5}
2,logistic_regression_model,0.980000,{'C': 10}
